In [1]:
# This R environment comes with all of CRAN preinstalled, as well as many other helpful packages
# The environment is defined by the kaggle/rstats docker image: https://github.com/kaggle/docker-rstats
# For example, here's several helpful packages to load in 

library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function
library(data.table)
library(Matrix)
library(xgboost)
library(Metrics)

ID = 'id'
TARGET = 'loss'
SEED = 0

TRAIN_FILE = "../input/train.csv"
TEST_FILE = "../input/test.csv"
SUBMISSION_FILE = "../input/sample_submission.csv"

train = fread(TRAIN_FILE, showProgress = TRUE)
test = fread(TEST_FILE, showProgress = TRUE)

y_train = log(train[,TARGET, with = FALSE])[[TARGET]]

train[, c(ID, TARGET) := NULL]
test[, c(ID) := NULL]

ntrain = nrow(train)
train_test = rbind(train, test)

features = names(train)

nbins = 10
binit = function(x,nbins=10){
  bins = unique(quantile(x,probs = seq(from=0, to=1, length.out=nbins)))
  bins[1] = bins[1] - 0.01
  output = cut(x, bins)
  levels = unique(output)
  output = as.integer(factor(output, levels=levels))
  return(output)
}

for (f in features) {
  falt = paste(f,"bin",sep="")
  if (class(train_test[[f]])=="character") {
    #cat("VARIABLE : ",f,"\n")
    levels <- unique(train_test[[f]])
    train_test[[f]] <- as.integer(factor(train_test[[f]], levels=levels))
  }
  else {
    train_test[[falt]] <- binit(train_test[[f]])
  }
}


x_train = train_test[1:ntrain,]
x_test = train_test[(ntrain+1):nrow(train_test),]


dtrain = xgb.DMatrix(as.matrix(x_train), label=y_train)
dtest = xgb.DMatrix(as.matrix(x_test))

Read 188318 rows and 132 (of 132) columns from 0.065 GB file in 00:00:04


cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,⋯,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
A,B,A,B,A,A,A,A,B,A,⋯,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
A,B,A,A,A,A,A,A,B,B,⋯,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
A,B,A,A,B,A,A,A,B,B,⋯,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
B,B,A,B,A,A,A,A,B,A,⋯,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
A,B,A,B,A,A,A,A,B,B,⋯,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606
A,B,A,A,A,A,A,A,B,A,⋯,0.302678,0.364464,0.401162,0.26847,0.46226,0.50556,0.366788,0.359249,0.345247,0.726792
A,A,A,A,B,A,A,A,A,A,⋯,0.295397,0.381515,0.363768,0.24564,0.40455,0.47225,0.334828,0.352251,0.342239,0.382931
A,B,A,B,A,A,A,A,B,A,⋯,0.473767,0.867021,0.583389,0.90267,0.84847,0.80218,0.644013,0.785706,0.859764,0.242416
A,B,B,B,B,A,A,A,B,B,⋯,0.281143,0.628534,0.384099,0.61229,0.38249,0.51111,0.682315,0.669033,0.756454,0.361191
A,B,A,A,B,B,A,A,B,A,⋯,0.310061,0.713343,0.469223,0.30260,0.67135,0.83510,0.863052,0.879347,0.822493,0.294523


cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,⋯,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
A,B,A,A,A,A,A,A,B,A,⋯,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
A,B,A,B,A,A,A,A,B,A,⋯,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
A,B,A,B,B,A,B,A,B,B,⋯,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
A,A,A,A,B,A,A,A,A,A,⋯,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
B,A,A,A,A,B,A,A,A,A,⋯,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823
A,A,A,A,B,A,A,A,A,A,⋯,0.643315,0.407351,0.390540,0.46477,0.46853,0.50556,0.607500,0.594646,0.250991,0.283976
B,A,A,A,B,B,A,A,A,A,⋯,0.281143,0.960845,0.740081,0.75964,0.98330,0.82249,0.863052,0.879347,0.888944,0.787807
B,B,A,A,A,A,A,A,B,A,⋯,0.651246,0.451115,0.316313,0.27320,0.52100,0.50556,0.415029,0.481306,0.199940,0.450597
A,B,A,A,A,A,A,A,B,A,⋯,0.534484,0.343492,0.358758,0.81900,0.32128,0.36458,0.453334,0.443374,0.695650,0.295075
A,B,A,A,A,A,A,A,B,B,⋯,0.281143,0.394921,0.287416,0.92347,0.48320,0.24766,0.359572,0.352251,0.519989,0.602666


In [2]:


xgb_params = list(
  seed = 0,
  colsample_bytree = 0.7,
  subsample = 0.7,
  eta = 0.075,
  objective = 'reg:linear',
  max_depth = 6,
  num_parallel_tree = 1,
  min_child_weight = 1,
  base_score = 7
)

xg_eval_mae <- function (yhat, dtrain) {
   y = getinfo(dtrain, "label")
   err= mae(exp(y),exp(yhat) )
   return (list(metric = "error", value = err))
}

res = xgb.cv(xgb_params,
             dtrain,
#             nrounds=750,
             nrounds=50,
             nfold=4,
             early_stopping_rounds=15,
             print_every_n = 10,
             verbose= 1,
             feval=xg_eval_mae,
             maximize=FALSE)

best_nrounds = res$best_iteration
cv_mean = res$evaluation_log$test_error_mean[best_nrounds]
cv_std = res$evaluation_log$test_error_std[best_nrounds]
cat(paste0('CV-Mean: ',cv_mean,' ', cv_std))

gbdt = xgb.train(xgb_params, dtrain, best_nrounds)

submission = fread(SUBMISSION_FILE, colClasses = c("integer", "numeric"))
submission$loss = exp(predict(gbdt,dtest))


[1]	train-error:2034.331151+5.502928	test-error:2034.366295+17.632346 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 15 rounds.

[11]	train-error:1649.772198+4.466486	test-error:1651.685043+17.611853 
[21]	train-error:1418.051092+5.161537	test-error:1423.303641+15.971279 
[31]	train-error:1300.917909+5.925941	test-error:1310.691646+14.295242 
[41]	train-error:1240.883830+5.450989	test-error:1254.567461+14.488630 
[50]	train-error:1209.543414+5.082898	test-error:1226.410298+14.668410 
CV-Mean: 1226.41029796687 14.6684104813553

In [3]:
str(submission)
head(submission)

write.csv(submission,'wk rgb faron1 1225.csv',row.names = FALSE)

Classes ‘data.table’ and 'data.frame':	125546 obs. of  2 variables:
 $ id  : int  4 6 9 12 15 17 21 28 32 43 ...
 $ loss: num  1557 1973 8022 4724 1028 ...
 - attr(*, ".internal.selfref")=<externalptr> 


id,loss
4,1556.920
6,1972.748
9,8021.522
12,4723.680
15,1028.181
17,1930.757
